# Section 2

## This is to ensure you have the proper tools installed

In [1]:
%pip install pandas
%pip install mysql-connector-python
%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## This checks if the necessary files are accessible and available

If the csv file can not be found download here and make sure it is in the correct directory:

https://drive.google.com/file/d/1B0P20CU9JP1I-jj0sgnq4QRvrtRLgUxF/view?usp=drive_link

In [2]:
from pathlib import Path 
import os

csv_file = "IMDB Movie Big Dataset.csv"
current_dir = Path(os.getcwd())

if (current_dir / csv_file).is_file():
    csv_file = (current_dir / csv_file)
    print("CSV present")
else:
    import kagglehub
    cached_path = Path(kagglehub.dataset_download("shubhamchandra235/imdb-and-tmdb-movie-metadata-big-dataset-1m"))
    if cached_path.is_file():
        csv_file = cached_path
    elif cached_path.is_dir():
        csv_file = list(cached_path.glob("*.csv"))[0]
        target_dir = str(current_dir) + "/IMDB Movie Big Dataset.csv"
        os.rename(csv_file, target_dir)
        csv_file = target_dir
        print("Successfully downloaded and moved file")

CSV present


In [71]:
import ast
import sqlite3
import pandas as pd

def col_to_list(col):
    if pd.isna(col) or col == '':
        return []
    col = str(col).strip()
    try:
        col_list = ast.literal_eval(col)
        if isinstance(col_list, list):
            return col_list
        else:
            return [str(col_list).strip()]
    except (ValueError, SyntaxError):
        return [i.strip() for i in col.split(',') if i.strip()]

# Turns columnns from string representation to array 

#Small version for testing queries
#df = pd.read_csv(csv_file, nrows=100)

#Full db
df = pd.read_csv(csv_file)

# Renames variables to better names
df.rename(columns={
    'AverageRating':'average_rating',
    'genres_list':'genres',
}, inplace=True)
df.columns = df.columns.str.lower()

column_list = [
    "production_companies",
    "production_countries",
    "spoken_languages",
    "keywords",
    "producers",
    "genres",
    "cast_list",
    "all_combined_keywords"
]

for l in column_list:
    df[l] = df[l].apply(col_to_list)

conn = sqlite3.connect("IMDB Dataset.sqlite")
df_main = df.drop(columns=column_list)
df_main.to_sql("movies", conn, if_exists="replace", index=False)

<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal


1072255

In [72]:
def create_junction_tables(df, col_id, col_list):
    junctions = {}
    for col in col_list:
        junction_df = (
            df[[col_id, col]]
                .explode(col)
                .dropna(subset=[col])
                .rename(columns={col_id: "movie_id", col:col})
        )
        junctions[col] = junction_df
    return junctions

junctions = create_junction_tables(df, col_id="id", col_list=column_list)

for col, junction_df in junctions.items():
    table_name = f"movie_{col}"
    junction_df.to_sql(table_name, conn, if_exists="replace", index=False)

### TEMP to show how to access the SQLite database

In [37]:
# TEMP to show how to access movies
conn = sqlite3.connect("IMDB Dataset.sqlite")
movies_df = pd.read_sql("SELECT * FROM movies", conn)
movies_df.head(50)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,imdb_rating,meta_score,star1,star2,star3,star4,writer,director_of_photography,music_composer,overview_sentiment
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,0,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,8.8,74.0,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,Christopher Nolan,Wally Pfister,Hans Zimmer,-0.011111
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,0,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,8.6,74.0,Matthew McConaughey,Anne Hathaway,Jessica Chastain,Mackenzie Foy,Jonathan Nolan,Hoyte van Hoytema,Hans Zimmer,0.045455
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,0,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,9.0,84.0,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,Jonathan Nolan,Wally Pfister,"Hans Zimmer, James Newton Howard",0.025000
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,0,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,7.8,83.0,Sam Worthington,Zoe Saldana,Sigourney Weaver,Michelle Rodriguez,James Cameron,Mauro Fiore,James Horner,0.041667
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,0,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,8.0,69.0,Robert Downey Jr.,Chris Evans,Scarlett Johansson,Jeremy Renner,Sydney Newman,NaN,NaN,0.025000
5,293660,Deadpool,7.606,28894,Released,2016-02-09,783100000,108,0,/en971MEXui9diirXlogOrPKmsEn.jpg,...,8.0,65.0,Ryan Reynolds,Morena Baccarin,T.J. Miller,Ed Skrein,Joss Whedon,NaN,NaN,-0.030293
6,299536,Avengers: Infinity War,8.255,27713,Released,2018-04-25,2052415039,149,0,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg,...,8.4,68.0,Joe Russo,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Rhett Reese,NaN,NaN,0.087662
7,550,Fight Club,8.438,27238,Released,1999-10-15,100853753,139,0,/hZkgoQYus5vegHoetLkCJzb17zJ.jpg,...,8.8,66.0,Brad Pitt,Edward Norton,Meat Loaf,Zach Grenier,Christopher Markus,NaN,NaN,-0.215909
8,118340,Guardians of the Galaxy,7.906,26638,Released,2014-07-30,772776600,121,0,/uLtVbjvS1O7gXL8lUOwsFOH4man.jpg,...,8.0,76.0,Chris Pratt,Vin Diesel,Bradley Cooper,Zoe Saldana,Chuck Palahniuk,Roger Pratt,Joel McNeely,0.400000
9,680,Pulp Fiction,8.488,25893,Released,1994-09-10,213900000,154,0,/suaEOtk1N1sgg2MTM7oZd2cfVp3.jpg,...,8.9,94.0,John Travolta,Uma Thurman,Samuel L. Jackson,Bruce Willis,James Gunn,Seamus McGarvey,Alan Silvestri,0.166667


In [35]:
# TEMP TO show how to access genres
genres_df = pd.read_sql("SELECT * FROM movie_genres", conn)
genres_df.head(100)

,movie_id,genres
0,27205,Action
1,27205,Science Fiction
2,27205,Adventure
3,157336,Adventure
4,157336,Drama
...,...,...
95,672,Adventure
96,672,Fantasy
97,283995,Science Fiction
98,283995,Adventure


In [60]:
movie_genres_df = pd.read_sql("""
SELECT id, title, vote_average, genres as genre
FROM movies  
INNER JOIN movie_genres ON (movies.id = movie_genres.movie_id)"""
, conn)
    
movie_genres_df.head(15)

,id,title,vote_average,genre
0,27205,Inception,8.364,Action
1,27205,Inception,8.364,Adventure
2,27205,Inception,8.364,Science Fiction
3,157336,Interstellar,8.417,Adventure
4,157336,Interstellar,8.417,Drama
5,157336,Interstellar,8.417,Science Fiction
6,155,The Dark Knight,8.512,Action
7,155,The Dark Knight,8.512,Crime
8,155,The Dark Knight,8.512,Drama
9,155,The Dark Knight,8.512,Thriller


In [ ]:
movie_genres_df = pd.read_sql("""
SELECT genres as genre, COUNT(*) as Occurances
FROM movie_genres  
GROUP BY genres
ORDER BY Occurances DESC"""
, conn)
movie_genres_df.head(15)

In [77]:
movie_genres_df = pd.read_sql("""
SELECT Count(genres) as "Num in Genre", AVG(vote_average) as "Average Score", AVG(revenue) as "Average Revenue", AVG(budget) as "Average Budget", genres as Genre
FROM movies  
INNER JOIN movie_genres ON (movies.id = movie_genres.movie_id)
WHERE vote_count > 100
GROUP BY genres
ORDER BY "Num in Genre" DESC """
, conn)
    
movie_genres_df

,Num in Genre,Average Score,Average Revenue,Average Budget,Genre
0,7871,6.750782,2.554977e+07,9.937910e+06,Drama
1,6554,6.375399,3.431879e+07,1.197755e+07,Comedy
2,4220,6.279822,3.593938e+07,1.480472e+07,Thriller
3,3638,6.390812,7.831901e+07,2.828512e+07,Action
4,2915,6.627981,3.199309e+07,1.030597e+07,Romance
5,2622,5.975515,1.712381e+07,5.951914e+06,Horror
6,2437,6.533073,1.169069e+08,3.730913e+07,Adventure
7,2430,6.568885,3.209203e+07,1.393097e+07,Crime
8,1848,6.598706,7.166163e+07,2.367613e+07,Family
9,1821,6.296111,7.726298e+07,2.690239e+07,Science Fiction


## Naïve Implementetation